In [1]:
%run "..\Startup_py3.py"
sys.path.append(r"..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# other required parameters
from ImageAnalysis3.classes import _allowed_kwds

22696


In [2]:
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

## load all adaptors

In [3]:
# find adaptors
reload(library_tools.sequences)
#library_tools.sequences.fasta_reader()

adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'
adaptor_files = [os.path.join(adaptor_folder, _fl) for _fl in os.listdir(adaptor_folder) 
                 if _fl.split(os.extsep)[-1]=='fasta' and _fl.split(os.extsep)[-2][-9:] == '_adaptors']
adaptors = library_tools.sequences.fasta_reader(adaptor_files, True).load()

loading 3 fasta files
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors\20200121_extend_stv_adaptors.fasta
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors\NDB_adaptors.fasta
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors\Stv_adaptors.fasta


In [10]:
os.listdir(adaptor_folder)

['20200121_extend_stv_adaptors.csv',
 '20200121_extend_stv_adaptors.fasta',
 '20210203_adaptor_telomere.fasta',
 'CTP-08_swap_cy3_adaptor.csv',
 'NDB_adaptor.csv',
 'NDB_adaptors.fasta',
 'Readout_sites.fasta',
 'Stv_adaptor.csv',
 'Stv_adaptors.fasta']

In [14]:
common_rd_file = os.path.join(adaptor_folder, 'Readout_sites.fasta')
common_rd_sites = library_tools.sequences.fasta_reader(common_rd_file, True).load()

loading 1 fasta files
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors\Readout_sites.fasta


In [15]:
common_rd_sites

[SeqRecord(seq=Seq('TTTGCACTGCCGTCCTTGAC'), id='Stv_82rc', name='Stv_82rc', description='Stv_82rc cy7 rev-com_last20', dbxrefs=[]),
 SeqRecord(seq=Seq('GATCCGATTGGAACCGTCCC'), id='Stv_1rc', name='Stv_1rc', description='Stv_1rc cy5 rev-com_last20', dbxrefs=[]),
 SeqRecord(seq=Seq('TGCGAACTGTCCGGCTTTCA'), id='Stv_79rc', name='Stv_79rc', description='Stv_79rc cy3 rev-com_last20', dbxrefs=[])]

In [4]:
len(adaptors)

1364

In [19]:
twocolor_adaptors = []
for _adaptor in adaptors:
    if "Stv" == _adaptor.id[:3] and ('Stv_82' in _adaptor.id or 'Stv_1' in _adaptor.id.split('2x')[1]):
        _rd_site = _adaptor.seq[:20]
        _seq = _rd_site+common_rd_sites[0].seq+common_rd_sites[1].seq
        _name = '_'.join([_adaptor.id.split('_2x')[0], common_rd_sites[0].id, common_rd_sites[1].id])
        _twocolor_adt = SeqRecord(_seq, id=_name, name='', description='')
        twocolor_adaptors.append(_twocolor_adt)

In [21]:
# generate csv file to order in IDT
import csv

with open(os.path.join(adaptor_folder, f'2color_Stv_adaptors.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\n',
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _adaptor in twocolor_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)

In [8]:

adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'

stv_adaptors = []

with open(os.path.join(adaptor_folder, 'Stv_adaptors.fasta'), 'r') as _rd_handle:
    for _adaptor in SeqIO.parse(_rd_handle, "fasta"):
        stv_adaptors.append(_adaptor)


In [13]:
stv_adaptors[0][:-20].reverse_complement()

SeqRecord(seq=Seq('GTCAAGGACGGCAGTGCAAACCCATGATCGTCCGATCTGG'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [14]:
stv_adaptors[0].id

'Stv_3_2xStv_82'

In [15]:
stv_adaptors[25][:-20].reverse_complement()

SeqRecord(seq=Seq('GGGACGGTTCCAATCGGATCCGAAGTACGCGGTACGAACG'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [16]:
stv_adaptors[25].id

'Stv_32_2xStv_1'